In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import pandas as pd
#from openai import OpenAI
import openai
import re

In [7]:
df = pd.read_csv('DF_pilot_withCaption_Foil_Idefics.csv', index_col = 0)
df

,ID_Video,Category,Annotator,Question,Answer,Caption,Foil,Idefics_Answer
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.,The absence of a mobile phone negates the poss...,The presence of a mobile phone facilitates th...,"No, the girl is not texting a message with her..."
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...,Stay safe during earthquakes by taking cover ...,Risk your safety during earthquakes by standi...,"If there was an earthquake, the computer monit..."
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say,The girl appears unsure while using the computer.,The girl seems confident and competent while ...,"No, the girl is not using the computer for the..."
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside,The weather outside is unclear.,The weather outside is very clear and sunny.,The weather is sunny.
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied,The girl appears pleased with the woman's stat...,The girl seems dissatisfied with the woman's ...,The girl looked at the woman in disbelief.
...,...,...,...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.,The resilient boy rises again after taking a h...,The fragile boy never gets up again after tak...,The boy with the axe falls to the ground.
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.,The boy with the sword strikes the boy with th...,The girl with the sword strikes the girl with...,The boy with the sword manages to hit the boy ...
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.,The victorious boy with the axe celebrates aft...,The defeated boy with the sword mourns after ...,The boy with the sword and the boy with the ax...
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.,The guys are playing.,The guys are not playing.,"Yes, the guys are playing."


In [ ]:
openai.api_key = "ca72e43668824cd090e6ea210fe3cd3b"
openai.api_base = "https://hlt-nlp.openai.azure.com/" # your endpoint should look like the following
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
deployment_name='gpt-35-turbo'



def GPT_answer_eval(System_answer, answer):
    response = openai.ChatCompletion.create(
        engine= deployment_name, # The deployment name you chose when you deployed the GPT-3.5-Turbo or GPT-4 model.
        messages=[
            {"role": "system", "content": f"You are an assistant designed to assess the similarity of sentences." 
            f"Users will paste in a string of text both the caption of a video, defined as C, and a declarative sentence S. "
            f"Given C and S, you must check whether they are semantically similar and whether they are expressing the same concept."
            f"If so, you must answer \"Correct\", otherwise type \"Not matching\". "},
            {"role": "user", "content": f"C: {System_answer}\nS: {answer}"}
        ]
    )
    return response['choices'][0]['message']['content']